# 0722 목표
- 가수별 list에 song 추가하기~

# 1. import libraries & data

In [1]:
# import libraries
import json
import re # regular expression
import gc # to collect memory
from datetime import datetime

import tensorflow as tf
import pandas as pd
import numpy as np

# for NLP
# from konlpy.tag import Kkma
# from konlpy.utils import pprint
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

import matplotlib.pyplot as plt
import seaborn as sns
# import missingno as msno

plt.style.use('seaborn')
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)

import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
font_name = matplotlib.font_manager.FontProperties(fname = 'c:/Windows/Fonts/malgun.ttf').get_name()
matplotlib.rc('font', family = font_name)

C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

In [2]:
# load json files
with open("./res/genre_gn_all.json", encoding = 'utf-8') as json_file: genre_gn_all = json.load(json_file)
with open("./res/song_meta.json", encoding = 'utf-8') as json_file: song_meta = json.load(json_file)
#with open("./res/train.json", encoding = 'utf-8') as json_file: train = json.load(json_file)
with open("./res/val.json", encoding = 'utf-8') as json_file: valid = json.load(json_file)

# json to dataframe
genre_gn_all = pd.Series(genre_gn_all).to_frame('genre').reset_index().rename({'index' : 'gen_id'}, axis = 1)
song_meta = pd.DataFrame(song_meta)
#train = pd.DataFrame(train)
train=pd.read_json('./res/train_topgnr.json', orient='table')
valid = pd.DataFrame(valid)

In [3]:
train
## most_appearing_gnr 추가되었음~~

,tags,id,plylst_title,songs,like_cnt,updt_date,most_appearing_gnr
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,GN0900
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,GN0100
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,GN0400
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000,GN0100
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000,GN0200
...,...,...,...,...,...,...,...
115066,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",120325,METAL E'SM #2,"[429629, 441511, 612106, 516359, 691768, 38714...",3,2020-04-17 04:31:11.000,GN1000
115067,[일렉],106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!,"[321330, 216057, 534472, 240306, 331098, 23288...",13,2015-12-24 17:23:19.000,GN1100
115068,"[담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]",11343,#1. 눈물이 앞을 가리는 나의_이야기,"[50512, 249024, 250608, 371171, 229942, 694943...",4,2019-08-16 20:59:22.000,GN0100
115069,"[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]",131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...",4,2019-10-25 23:40:42.000,GN1000


In [4]:
song_meta

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[Jude Law],4
...,...,...,...,...,...,...,...,...,...
707984,[GN2001],19991219,The Best Best Of The Black President,65254,[166499],Coffin For Head Of State,[GN2000],[Fela Kuti],707984
707985,[GN0901],19860000,True Colors,44141,[11837],Change Of Heart,[GN0900],[Cyndi Lauper],707985
707986,"[GN0105, GN0101]",20160120,행보 2015 윤종신 / 작사가 윤종신 Live Part.1,2662866,[437],스치듯 안녕,[GN0100],[윤종신],707986
707987,"[GN1807, GN1801]",20131217,명상의 시간을 위한 뉴에이지 음악,2221722,[729868],숲의 빛,[GN1800],[Nature Piano],707987


---

---
# 2. song_meta에 train에 등장하는 횟수 추가

In [5]:
# 플레이리스트 아이디(id)와 수록곡(songs) 추출
plylst_song_map = train[['id', 'songs']]

# unnest songs
plylst_song_map_unnest = np.dstack(
    (
        np.repeat(plylst_song_map.id.values, list(map(len, plylst_song_map.songs))), 
        np.concatenate(plylst_song_map.songs.values)
    )
)

# unnested 데이터프레임 생성 : plylst_song_map
plylst_song_map = pd.DataFrame(data = plylst_song_map_unnest[0], columns = plylst_song_map.columns)
plylst_song_map['id'] = plylst_song_map['id'].astype(str)
plylst_song_map['songs'] = plylst_song_map['songs'].astype(str)

# unnest 객체 제거
del plylst_song_map_unnest

In [6]:
plylst_song_map

,id,songs
0,61281,525514
1,61281,129701
2,61281,383374
3,61281,562083
4,61281,297861
...,...,...
5285866,100389,111365
5285867,100389,51373
5285868,100389,640239
5285869,100389,13759


In [7]:
# 1. 곡 별 수록된 플레이리스트 개수 count 테이블 생성 : song_plylst_cnt
song_plylst_cnt = plylst_song_map.groupby('songs').id.nunique().reset_index(name = 'mapping_plylst_cnt')

# 2. 단일/중복 수록 구분 : 곡 별 수록된 플레이리스트가 한 개면 '단일 수록', 두 개 이상이면 '중복 수록'
song_plylst_cnt = song_plylst_cnt.assign(
    mapping_plylst_cnt_category = pd.cut(song_plylst_cnt['mapping_plylst_cnt'], [0, 1, np.inf], labels = ['단일 수록', '중복 수록'])
)

# 3. 단일/중복 수록 구분 별 곡 수 count 테이블 생성 : song_plylst_mapping_division
song_plylst_mapping_division = pd.DataFrame(song_plylst_cnt.groupby('mapping_plylst_cnt_category').count())

# 4. 3번 테이블에 비율 값 추가
song_plylst_mapping_division.rename(columns = {'songs' : '곡 수'}, inplace = True)
song_plylst_mapping_division.loc[:, '비율(%)'] = round(song_plylst_mapping_division['곡 수']/sum(song_plylst_mapping_division['곡 수'])*100, 2)
song_plylst_mapping_division = song_plylst_mapping_division.reset_index().rename(columns = {'mapping_plylst_cnt_category' : '중복 수록 여부'})
song_plylst_mapping_division[['중복 수록 여부', '곡 수', '비율(%)']]

,중복 수록 여부,곡 수,비율(%)
0,단일 수록,299028,48.61
1,중복 수록,316114,51.39


In [8]:
song_plylst_cnt

,songs,mapping_plylst_cnt,mapping_plylst_cnt_category
0,0,2,중복 수록
1,10,32,중복 수록
2,1000,11,중복 수록
3,100000,2,중복 수록
4,100001,2,중복 수록
...,...,...,...
615137,99993,1,단일 수록
615138,99995,3,중복 수록
615139,99996,2,중복 수록
615140,99997,1,단일 수록


In [9]:
song_plylst_cnt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 615142 entries, 0 to 615141
Data columns (total 3 columns):
 #   Column                       Non-Null Count   Dtype   
---  ------                       --------------   -----   
 0   songs                        615142 non-null  object  
 1   mapping_plylst_cnt           615142 non-null  int64   
 2   mapping_plylst_cnt_category  615142 non-null  category
dtypes: category(1), int64(1), object(1)
memory usage: 10.0+ MB


In [10]:
#'songs' 정렬을 위해 str을 int로
song_plylst_cnt['songs']=song_plylst_cnt['songs'].astype(int)
song_plylst_cnt=song_plylst_cnt.sort_values(by='songs',axis=0)
song_plylst_cnt=song_plylst_cnt[['songs','mapping_plylst_cnt']]

In [11]:
song_plylst_cnt

,songs,mapping_plylst_cnt
0,0,2
193061,3,8
289673,4,2
386098,5,5
482599,6,1
...,...,...
586858,707984,1
586859,707985,4
586860,707986,11
586861,707987,1


In [12]:
song_plylst_cnt['song']=song_plylst_cnt['songs']

In [13]:
song_plylst_cnt

,songs,mapping_plylst_cnt,song
0,0,2,0
193061,3,8,3
289673,4,2,4
386098,5,5,5
482599,6,1,6
...,...,...,...
586858,707984,1,707984
586859,707985,4,707985
586860,707986,11,707986
586861,707987,1,707987


결측값이 있다..

In [14]:
song_plylst_cnt=song_plylst_cnt.set_index('songs')
song_plylst_cnt

,mapping_plylst_cnt,song
songs,,
0,2,0
3,8,3
4,2,4
5,5,5
6,1,6
...,...,...
707984,1,707984
707985,4,707985
707986,11,707986


In [15]:
song_meta_cnt=pd.DataFrame()
song_meta_cnt=song_meta

#빈 열 만들기
song_meta_cnt['count']=pd.Series()
song_meta_cnt

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,count
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0,NaN
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1,NaN
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2,NaN
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3,NaN
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[Jude Law],4,NaN
...,...,...,...,...,...,...,...,...,...,...
707984,[GN2001],19991219,The Best Best Of The Black President,65254,[166499],Coffin For Head Of State,[GN2000],[Fela Kuti],707984,NaN
707985,[GN0901],19860000,True Colors,44141,[11837],Change Of Heart,[GN0900],[Cyndi Lauper],707985,NaN
707986,"[GN0105, GN0101]",20160120,행보 2015 윤종신 / 작사가 윤종신 Live Part.1,2662866,[437],스치듯 안녕,[GN0100],[윤종신],707986,NaN
707987,"[GN1807, GN1801]",20131217,명상의 시간을 위한 뉴에이지 음악,2221722,[729868],숲의 빛,[GN1800],[Nature Piano],707987,NaN


In [16]:
song_meta_cnt['count'][song_plylst_cnt['song']]=song_plylst_cnt['mapping_plylst_cnt']

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [17]:
song_meta_cnt

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,count
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0,2.0
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1,NaN
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2,NaN
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3,8.0
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[Jude Law],4,2.0
...,...,...,...,...,...,...,...,...,...,...
707984,[GN2001],19991219,The Best Best Of The Black President,65254,[166499],Coffin For Head Of State,[GN2000],[Fela Kuti],707984,1.0
707985,[GN0901],19860000,True Colors,44141,[11837],Change Of Heart,[GN0900],[Cyndi Lauper],707985,4.0
707986,"[GN0105, GN0101]",20160120,행보 2015 윤종신 / 작사가 윤종신 Live Part.1,2662866,[437],스치듯 안녕,[GN0100],[윤종신],707986,11.0
707987,"[GN1807, GN1801]",20131217,명상의 시간을 위한 뉴에이지 음악,2221722,[729868],숲의 빛,[GN1800],[Nature Piano],707987,1.0


In [18]:
#nan 대신에 0채우기
song_meta_cnt['count'][song_meta_cnt['count'].isnull()]=0.0
song_meta_cnt
#song_meta_cnt['count'].isnull()

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,count
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0,2.0
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1,0.0
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2,0.0
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3,8.0
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[Jude Law],4,2.0
...,...,...,...,...,...,...,...,...,...,...
707984,[GN2001],19991219,The Best Best Of The Black President,65254,[166499],Coffin For Head Of State,[GN2000],[Fela Kuti],707984,1.0
707985,[GN0901],19860000,True Colors,44141,[11837],Change Of Heart,[GN0900],[Cyndi Lauper],707985,4.0
707986,"[GN0105, GN0101]",20160120,행보 2015 윤종신 / 작사가 윤종신 Live Part.1,2662866,[437],스치듯 안녕,[GN0100],[윤종신],707986,11.0
707987,"[GN1807, GN1801]",20131217,명상의 시간을 위한 뉴에이지 음악,2221722,[729868],숲의 빛,[GN1800],[Nature Piano],707987,1.0


In [19]:
song_meta_cnt['count']=song_meta_cnt['count'].astype(int)
song_meta_cnt

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,count
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0,2
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1,0
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2,0
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3,8
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[Jude Law],4,2
...,...,...,...,...,...,...,...,...,...,...
707984,[GN2001],19991219,The Best Best Of The Black President,65254,[166499],Coffin For Head Of State,[GN2000],[Fela Kuti],707984,1
707985,[GN0901],19860000,True Colors,44141,[11837],Change Of Heart,[GN0900],[Cyndi Lauper],707985,4
707986,"[GN0105, GN0101]",20160120,행보 2015 윤종신 / 작사가 윤종신 Live Part.1,2662866,[437],스치듯 안녕,[GN0100],[윤종신],707986,11
707987,"[GN1807, GN1801]",20131217,명상의 시간을 위한 뉴에이지 음악,2221722,[729868],숲의 빛,[GN1800],[Nature Piano],707987,1


---
# 3. 가수 list 만들기

In [20]:
#담을 곳 선언
artist_list=pd.DataFrame()

#artist id와 name
#song_meta['artist_id_basket']
#song_meta['artist_name_basket']

# 첫 번째것만 담겠다. 왜냐하면 나중에 딕셔너리는 정렬이 안됨ㅠ
artist_list['id']=song_meta['artist_id_basket'].str[0] #list 첫번째 항목만 넣는다
artist_list['name']=song_meta['artist_name_basket'].str[0] #list 첫번째 항목만 넣는다

In [21]:
artist_list

,id,name
0,2727,Various Artists
1,29966,Murray Perahia
2,3361,Peter Gabriel
3,838543,Matoma
4,560160,Jude Law
...,...,...
707984,166499,Fela Kuti
707985,11837,Cyndi Lauper
707986,437,윤종신
707987,729868,Nature Piano


In [22]:
# artist id중 맨 앞에 있는 이름으로 정렬
artist_list=artist_list.sort_values(by=['id'],axis=0)
artist_list

,id,name
305875,1,나훈아
169859,1,나훈아
321108,1,나훈아
48843,1,나훈아
170749,1,나훈아
...,...,...
512947,2865590,라포
334050,2865591,펭수
442813,2866236,Midas Touch
318592,2866406,Moon Arch


In [23]:
artist_list=artist_list.drop_duplicates('id', keep="first")

#index reset하겠음(문자 단위 검색 목적이므로)
artist_list=artist_list.reset_index()
del artist_list['index']

artist_list

,id,name
0,1,나훈아
1,2,남화용
2,5,유현상
3,6,김학래
4,7,정태춘
...,...,...
103573,2865590,라포
103574,2865591,펭수
103575,2866236,Midas Touch
103576,2866406,Moon Arch


- id 3이나 4가 사라진게아니라 sont_meta data 내에는 없다..
- 아티스트 한 10만명 정도~

In [24]:
artistsong_list=pd.DataFrame()
artistsong_list=artist_list

#빈 열 만들기
artistsong_list['song_id']=pd.Series()

artistsong_list

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """


,id,name,song_id
0,1,나훈아,NaN
1,2,남화용,NaN
2,5,유현상,NaN
3,6,김학래,NaN
4,7,정태춘,NaN
...,...,...,...
103573,2865590,라포,NaN
103574,2865591,펭수,NaN
103575,2866236,Midas Touch,NaN
103576,2866406,Moon Arch,NaN


In [25]:
#빈배열 만들기ㅠㅠㅠ

#song_id 형을 변환해준다!
artistsong_list['song_id'][0]=['test']
artistsong_list['song_id'][0].pop()

for i in range(0,103578):
    artistsong_list['song_id'][i]=[]
artistsong_list

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\user\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,id,name,song_id
0,1,나훈아,[]
1,2,남화용,[]
2,5,유현상,[]
3,6,김학래,[]
4,7,정태춘,[]
...,...,...,...
103573,2865590,라포,[]
103574,2865591,펭수,[]
103575,2866236,Midas Touch,[]
103576,2866406,Moon Arch,[]


In [26]:
# 많이 나온 순으로 하기 위해서 song_meta_cnt 정렬후하겠음
## 많은 순으로 정렬~~~
song_meta_cnt=song_meta_cnt.sort_values(by=['count'],axis=0,ascending=False)
song_meta_cnt=song_meta_cnt.reset_index()
del song_meta_cnt['index']
song_meta_cnt

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,count
0,"[GN0105, GN0101]",20170324,밤편지,10047890,[261143],밤편지,[GN0100],[아이유],144663,2175
1,"[GN0501, GN0601, GN0503, GN0606, GN0509]",20111123,Lo9ve3r4s,2038488,[242988],안아줘,"[GN0500, GN0600]",[정준일],116573,2121
2,"[GN0401, GN0403]",20160621,비,2692501,[752425],비,[GN0400],[폴킴],357367,1981
3,"[GN0805, GN0509, GN0502, GN0801, GN0501]",20101007,가을방학,1035872,[437760],가끔 미치도록 네가 안고 싶어질 때가 있어,"[GN0500, GN0800]",[가을방학],366786,1919
4,"[GN1501, GN1504]",20041115,미안하다 사랑한다 OST,43841,[1191],눈의 꽃,[GN1500],[박효신],654757,1647
...,...,...,...,...,...,...,...,...,...,...
707984,[GN1001],20130813,Overnight,2223846,[422703],Waiting For That Call (Album Version),[GN1000],[Parachute],293017,0
707985,[GN0801],20000905,신형원 베스트 1982-2000,117,[2020],비오는 날의 가단조,[GN0800],[신형원],293019,0
707986,"[GN0601, GN0604]",19980101,Funky Station,7247,[1552],이탈(feat. 이현도),[GN0600],[한상원],113722,0
707987,"[GN1807, GN1801]",20150403,편안한 우리집에서 듣는 뉴에이지,198029,[730327],다시 돌아온 계절,[GN1800],[Valenpino],113724,0


In [27]:
song_info = pd.read_json('./song_info.json', orient='table')
song_info

,artist_id_basket,id,count,artist_len
0,[261143],144663,2175,1
1,[242988],116573,2121,1
2,[752425],357367,1981,1
3,[437760],366786,1919,1
4,[1191],654757,1647,1
...,...,...,...,...
707984,[1552],113722,0,1
707985,[730327],113724,0,1
707986,[883201],612320,0,1
707987,"[346714, 745335]",521169,0,2


In [28]:
import time
start = time.time()

for i in range(90001,92501):
    for j in range(0,707989):
        for k in range(0,song_info['artist_len'][j]):
            if (song_info['artist_id_basket'][j][k]==artistsong_list['id'][i]):
                artistsong_list['song_id'][i].append(song_info['id'][j])
    if(i%50==0):
        print('\n',i)
        print("time :", time.time() - start) 
        print("\n")
        
        #dataframe으로 중간저장
        artistsong_list.to_json('./artistsong_list_save90001.json', orient='table')
    else:
        print(i)
        print("time :", time.time() - start) 
        print("\n")
        
print("FINISH TIME!! :", time.time() - start) 

90001
time : 28.2248694896698


90002
time : 57.68444609642029


90003
time : 87.79657483100891


90004
time : 117.71171975135803


90005
time : 147.34801697731018


90006
time : 176.5456473827362


90007
time : 205.96715593338013


90008
time : 235.71127486228943


90009
time : 265.46054577827454


90010
time : 295.63729453086853


90011
time : 325.9659740924835


90012
time : 356.08178448677063


90013
time : 385.7634267807007


90014
time : 415.6228516101837


90015
time : 445.12678718566895


90016
time : 474.91867995262146


90017
time : 504.80051040649414


90018
time : 535.9034266471863


90019
time : 566.0681233406067


90020
time : 596.4091970920563


90021
time : 627.2516520023346


90022
time : 657.8819880485535


90023
time : 688.2030870914459


90024
time : 717.8591847419739


90025
time : 748.6082723140717


90026
time : 778.6045382022858


90027
time : 807.4218981266022


90028
time : 837.5089144706726


90029
time : 867.5791189670563


90030
time : 897.00221824646


900

90248
time : 8274.045438289642


90249
time : 8306.239831924438



 90250
time : 8341.664333343506


90251
time : 8372.992351770401


90252
time : 8402.458881378174


90253
time : 8434.597180128098


90254
time : 8464.782638549805


90255
time : 8494.217143535614


90256
time : 8524.21650314331


90257
time : 8553.333860874176


90258
time : 8585.156977176666


90259
time : 8615.410099983215


90260
time : 8644.743953227997


90261
time : 8675.825016975403


90262
time : 8707.5195581913


90263
time : 8736.847195386887


90264
time : 8767.00898551941


90265
time : 8796.67402601242


90266
time : 8828.629331350327


90267
time : 8862.190805912018


90268
time : 8892.016422510147


90269
time : 8923.295069932938


90270
time : 8958.688056230545


90271
time : 8990.015583992004


90272
time : 9020.146323680878


90273
time : 9052.680595636368


90274
time : 9088.069714069366


90275
time : 9118.428656339645


90276
time : 9148.57103228569


90277
time : 9182.633101463318


90278
time : 9

KeyboardInterrupt: 

In [31]:
artistsong_list[90001:]

,id,name,song_id
90001,2140364,Haley & Michaels,[47360]
90002,2140365,KLIM,"[94214, 183914, 173407, 115138, 109375, 524661..."
90003,2140367,You In Mind,"[145960, 187588, 367545]"
90004,2140371,Niall Anderson,"[278387, 467328]"
90005,2140381,Bewitched,[460900]
...,...,...,...
103573,2865590,라포,[]
103574,2865591,펭수,[]
103575,2866236,Midas Touch,[]
103576,2866406,Moon Arch,[]


In [32]:
#dataframe으로 저장
artistsong_list.to_json('./artistsong_list90001_90036.json', orient='table')

In [35]:
test_data = pd.read_json('./artistsong_list90001_90036.json', orient='table')
test_data[90001:]

,id,name,song_id
90001,2140364,Haley & Michaels,[47360]
90002,2140365,KLIM,"[94214, 183914, 173407, 115138, 109375, 524661..."
90003,2140367,You In Mind,"[145960, 187588, 367545]"
90004,2140371,Niall Anderson,"[278387, 467328]"
90005,2140381,Bewitched,[460900]
...,...,...,...
103573,2865590,라포,[]
103574,2865591,펭수,[]
103575,2866236,Midas Touch,[]
103576,2866406,Moon Arch,[]


In [ ]:
song_meta[song_meta['id']==535182]